# Stochastic Signal Detection

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from pathlib import Path
import pandas as pd
import plotly as py
from plotly import express as px
from sqlalchemy import create_engine
from IPython.display import display, HTML

py.offline.init_notebook_mode(connected=True)
display(
    HTML(
        '<script type="text/javascript" id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"></script>'
    ))


## Visualization

We use SQLite and `pandas` to load the simulation results.

In [17]:
db = 'simulation.sqlite'
table = 'stochastic_signal_detection'
output = Path('visualization')
output.mkdir(exist_ok=True, parents=True)
sql = create_engine(f'sqlite:///{db}')
df = pd.read_sql_table(table, sql)

The table groups various variables of the simulation:

In [18]:
df.describe()

,id,timestamp,seed,rows,ratio,rank,beta,mass_scale,mass_scale_bottom,mass_scale_top,...,T,nmax_chi_0,max_chi_0,argmax_chi_0,nmax_chi_1,max_chi_1,argmax_chi_1,min_frac,nmin_frac,argmin_frac
count,2007.000000,2007,2007.000000,2007.0,2.007000e+03,2007.0,2007.000000,2007.000000,2007.000000,2007.000000,...,2007.000000,2007.000000,2.007000e+03,2007.000000,2007.000000,2.007000e+03,2007.000000,2007.000000,2007.000000,2007.000000
mean,1004.000000,2023-09-15 15:21:53.841056256,16447.670653,10000.0,8.000000e-01,2500.0,0.262668,0.055468,0.043905,0.071817,...,0.785625,403.813154,5.271825e+48,0.239934,488.986049,6.109621e+295,0.251344,-22.694810,71.938714,0.174704
min,1.000000,2023-09-15 12:46:59,2.000000,10000.0,8.000000e-01,2500.0,0.000000,0.000000,0.000000,0.014985,...,0.000000,5.000000,-1.199519e+47,0.073732,19.000000,-8.755503e+283,0.122481,-4784.851924,0.000000,0.000000
25%,502.500000,2023-09-15 14:04:03.500000,8381.000000,10000.0,8.000000e-01,2500.0,0.150000,0.015830,0.001179,0.032241,...,0.375000,375.000000,-2.671836e+03,0.148124,497.000000,-4.595600e+15,0.179627,-6.182259,4.000000,0.039939
50%,1004.000000,2023-09-15 15:21:49,16376.000000,10000.0,8.000000e-01,2500.0,0.300000,0.059637,0.044877,0.075940,...,0.775000,500.000000,-1.523333e+00,0.230295,499.000000,4.165014e+00,0.273871,-0.854311,15.000000,0.214295
75%,1505.500000,2023-09-15 16:39:18.500000,24349.500000,10000.0,8.000000e-01,2500.0,0.412500,0.097470,0.082769,0.113896,...,1.175000,500.000000,2.859837e+00,0.319969,500.000000,2.351212e+13,0.321170,0.314053,51.000000,0.307261
max,2007.000000,2023-09-15 17:56:55,32732.000000,10000.0,8.000000e-01,2500.0,0.525000,0.131103,0.116343,0.147642,...,1.575000,501.000000,1.058060e+52,0.384242,501.000000,1.226201e+299,0.383814,0.999885,501.000000,0.383814
std,579.515315,NaN,9422.161602,0.0,1.110500e-16,0.0,0.171734,0.045754,0.042285,0.045868,...,0.462064,171.514035,2.361765e+50,0.090323,53.396155,inf,0.089179,166.030052,137.324088,0.131900


In particular, the final columns represent the "measure of criticality" of the simulation:

- `nmax_chi_0` is the position of the maximum of the derivative of the potential in the list of values of $k$ used for the simulation for $\chi = 0$, i.e.: $n = \mathrm{pos}\max\left|\mathcal{U}'[0]\right|$, where $n \in [0, 500]$;
- `argmax_chi_0` is the position at the maximum of the derivative for $\chi = 0$, i.e.: $\overline{k} = \arg\max\left|\mathcal{U}'[0]\right|$;
- `max_chi_0` is the value at the maximum of the derivative for $\chi = 0$, i.e.: $\mathcal{U}'[0](\overline{k})$;
- `nmax_chi_1` is the position of the maximum of the derivative of the potential in the list of values of $k$ used for the simulation for $\chi = 1$, i.e.: $n = \mathrm{pos}\max\left|\mathcal{U}'[1]\right|$, where $n \in [0, 500]$;
- `argmax_chi_1` is the position at the maximum of the derivative for $\chi = 1$, i.e.: $\overline{k} = \arg\max\left|\mathcal{U}'[1]\right|$;
- `max_chi_1` is the value at the maximum of the derivative for $\chi = 1$, i.e.: $\mathcal{U}'[1](\overline{k})$.

### Scan in $\beta$

We first take a look at the results by grouping the simulations by $\beta$ (_signal-to-noise ratio_).

In [19]:
df_group = df.groupby('beta')
betas = df_group.groups.keys()

We explore the phase space of $\mu_{0}$, $\mu_{1}$ and $\mu_{2}$ for various values:

In [45]:
for beta in betas:
    df_beta = df_group.get_group(beta)
    fig = px.scatter_3d(
        df_beta,
        x='mu_0',
        y='mu_1',
        z='mu_2',
        color='nmax_chi_0',
        custom_data=['nmax_chi_0', 'T'],
        title=fr'$\beta = {beta}$',
    )
    fig.layout.coloraxis.colorbar.title = r'$n_{max}\mathcal{U}_{0}$'
    fig.update_layout(scene=dict(
        xaxis_title=r'$\mu_{0}$',
        yaxis_title=r'$\mu_{1}$',
        zaxis_title=r'$\mu_{2}$',
    ),
                      width=800,
                      height=800)
    fig.update_traces(
        hovertemplate=
        r'$\mu_{0}$: %{x:.2f}<br>$\mu_{1}$: %{y:.2f}<br>$\mu_{2}$: %{z}<br>$n_{max}\mathcal{U}_{0}$: %{customdata[0]}<br>T = %{customdata[1]}'
    )
    fig.write_html(str(output / f'beta_{beta}_nmax_chi_0.html'),
                   include_plotlyjs='cdn',
                   include_mathjax='cdn')


In [40]:
for beta in betas:
    df_beta = df_group.get_group(beta)
    fig = px.scatter_3d(
        df_beta,
        x='mu_0',
        y='mu_1',
        z='mu_2',
        color='max_chi_0',
        custom_data=['max_chi_0', 'T'],
        title=fr'$\beta = {beta}$',
    )
    fig.layout.coloraxis.colorbar.title = r'$\max\left|\mathcal{U}_{0}\right|$'
    fig.update_layout(scene=dict(
        xaxis_title=r'$\mu_{0}$',
        yaxis_title=r'$\mu_{1}$',
        zaxis_title=r'$\mu_{2}$',
    ),
                      width=800,
                      height=800)
    fig.update_traces(
        hovertemplate=
        r'$\mu_{0}$: %{x:.2f}<br>$\mu_{1}$: %{y:.2f}<br>$\mu_{2}$: %{z}<br>$\max\left|\mathcal{U}_{0}\right|$: %{customdata[0]}<br>T = %{customdata[1]}'
    )
    fig.write_html(str(output / f'beta_{beta}_max_chi_0.html'),
                   include_plotlyjs='cdn',
                   include_mathjax='cdn')


In [41]:
for beta in betas:
    df_beta = df_group.get_group(beta)
    fig = px.scatter_3d(
        df_beta,
        x='mu_0',
        y='mu_1',
        z='mu_2',
        color='nmax_chi_1',
        custom_data=['nmax_chi_1', 'T'],
        title=fr'$\beta = {beta}$',
    )
    fig.layout.coloraxis.colorbar.title = r'$n_{max}\mathcal{U}_{1}$'
    fig.update_layout(scene=dict(
        xaxis_title=r'$\mu_{0}$',
        yaxis_title=r'$\mu_{1}$',
        zaxis_title=r'$\mu_{2}$',
    ),
                      width=800,
                      height=800)
    fig.update_traces(
        hovertemplate=
        r'$\mu_{0}$: %{x:.2f}<br>$\mu_{1}$: %{y:.2f}<br>$\mu_{2}$: %{z}<br>$n_{max}\mathcal{U}_{1}$: %{customdata[0]}<br>T = %{customdata[1]}'
    )
    fig.write_html(str(output / f'beta_{beta}_nmax_chi_1.html'),
                   include_plotlyjs='cdn',
                   include_mathjax='cdn')


In [42]:
for beta in betas:
    df_beta = df_group.get_group(beta)
    fig = px.scatter_3d(
        df_beta,
        x='mu_0',
        y='mu_1',
        z='mu_2',
        color='max_chi_1',
        custom_data=['max_chi_1', 'T'],
        title=fr'$\beta = {beta}$',
    )
    fig.layout.coloraxis.colorbar.title = r'$\max\left|\mathcal{U}_{1}\right|$'
    fig.update_layout(scene=dict(
        xaxis_title=r'$\mu_{0}$',
        yaxis_title=r'$\mu_{1}$',
        zaxis_title=r'$\mu_{2}$',
    ),
                      width=800,
                      height=800)
    fig.update_traces(
        hovertemplate=
        r'$\mu_{0}$: %{x:.2f}<br>$\mu_{1}$: %{y:.2f}<br>$\mu_{2}$: %{z}<br>$\max\left|\mathcal{U}_{1}\right|$: %{customdata[0]}<br>T = %{customdata[1]}'
    )
    fig.write_html(str(output / f'beta_{beta}_max_chi_1.html'),
                   include_plotlyjs='cdn',
                   include_mathjax='cdn')


### Scan in $T$

We then look at a scan in $T$, the temperature, for different values of $\beta$.

In [24]:
df_group = df.groupby('T')
Ts = df_group.groups.keys()

We explore the phase space of $\beta$:

In [25]:
for t in Ts:
    df_t = df_group.get_group(t).groupby('beta').mean().reset_index()
    fig = px.line(
        df_t,
        x='beta',
        y='nmax_chi_0',
        title=fr'$T = {t}$',
        markers=True,
        labels=dict(
            beta=r'$\beta$',
            nmax_chi_0=r'$n_{max}\mathcal{U}_{0}$',
        ),
    )
    fig.update_traces(
        marker=dict(size=10),
        hovertemplate=r'$\beta$: %{x:.2f}<br>$n_{max}\mathcal{U}_{0}$: %{y:.2f}'
    )
    fig.write_html(str(output / f'T_{t}_nmax_chi_0.html'),
                   include_plotlyjs='cdn',
                   include_mathjax='cdn')


In [26]:
for t in Ts:
    df_t = df_group.get_group(t).groupby('beta').mean().reset_index()
    fig = px.line(
        df_t,
        x='beta',
        y='nmax_chi_1',
        title=fr'$T = {t}$',
        markers=True,
        labels=dict(
            beta=r'$\beta$',
            nmax_chi_1=r'$n_{max}\mathcal{U}\'\[1\]$',
        ),
    )
    fig.update_traces(
        marker=dict(size=10),
        hovertemplate=r'$\beta$: %{x:.2f}<br>$n_{max}\mathcal{U}_{1}$: %{y:.2f}'
    )
    fig.write_html(str(output / f'T_{t}_nmax_chi_1.html'),
                   include_plotlyjs='cdn',
                   include_mathjax='cdn')


In [27]:
for t in Ts:
    df_t = df_group.get_group(t).groupby('beta').mean().reset_index()
    fig = px.line(
        df_t,
        x='beta',
        y='max_chi_0',
        title=fr'$T = {t}$',
        markers=True,
        labels=dict(
            beta=r'$\beta$',
            max_chi_0=r'$\max\left|\mathcal{U}_{0}$\right|',
        ),
    )
    fig.update_traces(
        marker=dict(size=10),
        hovertemplate=
        r'$\beta$: %{x:.2f}<br>$\max\left|\mathcal{U}\'\[0\]\right|$: %{y:.2f}')
    fig.write_html(str(output / f'T_{t}_max_chi_0.html'),
                   include_plotlyjs='cdn',
                   include_mathjax='cdn')


In [28]:
for t in Ts:
    df_t = df_group.get_group(t).groupby('beta').mean().reset_index()
    fig = px.line(
        df_t,
        x='beta',
        y='max_chi_1',
        title=fr'$T = {t}$',
        markers=True,
        labels=dict(
            beta=r'$\beta$',
            max_chi_1=r'$\max\left|\mathcal{U}_{1}$\right|',
        ),
    )
    fig.update_traces(
        marker=dict(size=10),
        hovertemplate=
        r'$\beta$: %{x:.2f}<br>$\max\left|\mathcal{U}_{1}\right|$: %{y:.2f}')
    fig.write_html(str(output / f'T_{t}_max_chi_1.html'),
                   include_plotlyjs='cdn',
                   include_mathjax='cdn')
